# Project 3. E.coli outbreak investigation. De novo assembly and annotation of bacterial genomes.

Lab notebook by Nikita Vyatkin

This week, you will follow in the footsteps of the bioinformaticians investigating the outbreak by assembling the genome of the deadly E. coli X strain. Specifically, we will provide you with Illumina reads from the TY2482 sample, which were generated at Beijing Genome Institute and deposited into the Short Read Archive (http://www.ncbi.nlm.nih.gov/sra) for public access. After we have assembled the E. coli X genome, we will determine which other strain(s) it is most similar to in an effort to determine how it could have arisen. In particular, once we know where it came from, we can ask what new genes it possesses for pathogenicity, and how it evolved these new functions.

Our workflow will be to answer the following questions:
1. What is the genome sequence of E. coli X?
2. What strain of E. coli is E. coli X most similar to? (Where did it come from?)
3. What are the genes that E. coli X contains?
4. Which of these genes make E. coli X distinct?
5. How did E. coli X evolve to obtain these genes?
6. How did E. coli X become pathogenic?





## Assembling the Genome

DNA sequencing technology cannot read whole genomes in one go, so we will assemble genome from the short reads generated by the Illumina instrument. We will use the [SPAdes](https://cab.spbu.ru/software/spades/) assembler (Bankevich et al., 2012) to assemble the E. coli X genome from reads into contigs.

A contig is a “contiguous” segment of the genome that has been reconstructed by an assembly algorithm. In addition to contigs, SPAdes uses information about the distances between reads within read-pairs (called insert size) to combine contigs into ordered collections of adjacent contigs called scaffolds.Genome assembly programs often fill a gap in a scaffold of length m by a sequence of m occurrences of “N” (a placeholder for unknown nucleotides).

In modern DNA sequencing projects, DNA fragments are sequenced from both the 5' and 3' ends, giving rise to paired reads separated by some insert size. The forward and reverse read in a paired read are generated from the forward and reverse strand, respectively. A collection of paired reads is called a sequencing library; whereas most paired libraries generate reads with insert sizes below 1 kb, libraries with longer (2 kb-10 kb) insert sizes are called mate-pairs.

Many sequencing projects generate several libraries of paired reads with different insert sizes; for example, the sequencing project at the center of this task employs three libraries of insert lengths 270, 2000, and 6000 nucleotides.

The advantage of using multiple libraries is that libraries with small insert sizes are better suited for resolving short repeats, whereas libraries with larger insert sizes are better suited for resolving long repeats. For example, all bacterial genomes contain ribosomal operons, which are often repeated more than six times and are approximately 5000 nucleotides long. Thus, to resolve ribosomal operons, we need mate-pair libraries with insert sizes exceeding 5000 nt.

Once we have sequenced a genome, there are many metrics for assessing the resulting assembly. We will use QUAST (Gurevich et al., 2013), which takes many metrics into account to evaluate the overall quality of an assembly.

One of the common metrics is the N50 statistic. N50 is defined as the maximal contig length for which all contigs greater than or equal to that length comprise at least half of the sum of the lengths of all the contigs. For example, consider the five toy contigs with the following lengths: [10, 20, 30, 60, 70]. Here, the total length of contigs is 190, and contigs of length 60 and 70 account for at least 50% of the total length of contigs (60 + 70 = 130), but the contig of length 70 does not account for 50% of the total length of contigs. Thus, N50 is equal to 60.

## Dataset

For this project, we provide three libraries from the TY2482 sample with the following
insert sizes and orientation:
* SRR292678 - paired end, insert size 470 bp ([forward reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R1_001.fastq.gz), [reverse reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R2_001.fastq.gz), 400 Mb each)
* SRR292862 - mate pair, insert size 2 kb, ([forward reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292862_S2_L001_R1_001.fastq.gz), [reverse reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292862_S2_L001_R2_001.fastq.gz), 200 Mb each)
* SRR292770 - mate pair, insert size 6 kb, ([forward reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292770_S1_L001_R1_001.fastq.gz), [reverse reads](https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292770_S1_L001_R2_001.fastq.gz), 200 Mb each)

In [4]:
! mkdir raw_data

In [7]:
! wget -P raw_data https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R1_001.fastq.gz https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R2_001.fastq.gz https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292862_S2_L001_R1_001.fastq.gz https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292862_S2_L001_R2_001.fastq.gz https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292770_S1_L001_R1_001.fastq.gz https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292770_S1_L001_R2_001.fastq.gz

--2022-11-21 18:03:20--  https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R1_001.fastq.gz
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.33.244.215, 13.33.244.96, 13.33.244.69, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.33.244.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 425982530 (406M) [application/x-gzip]
Saving to: ‘raw_data/SRR292678sub_S1_L001_R1_001.fastq.gz’

SRR292678sub_S1_L00 100%[===================>] 406,25M  3,62MB/s    in 2m 1s   

2022-11-21 18:05:21 (3,37 MB/s) - ‘raw_data/SRR292678sub_S1_L001_R1_001.fastq.gz’ saved [425982530/425982530]

--2022-11-21 18:05:21--  https://d28rh4a8wq0iu5.cloudfront.net/bioinfo/SRR292678sub_S1_L001_R2_001.fastq.gz
Reusing existing connection to d28rh4a8wq0iu5.cloudfront.net:443.
HTTP request sent, awaiting response... 200 OK
Length: 426246191 (406M) [application/x-gzip]
Saving to: ‘raw_data/SRR292678sub_S1_L001_R2_001.

In [9]:
! gunzip raw_data/*

### Simple manual data analysis

In [4]:
out = ! wc -l raw_data/*.fastq

out = [s.split() for s in out]

for item in out:
    print(f"Number of reads in {item[1]}: {int(item[0]) // 4:,}")

Number of reads in raw_data/SRR292678sub_S1_L001_R1_001.fastq: 5,499,346
Number of reads in raw_data/SRR292678sub_S1_L001_R2_001.fastq: 5,499,346
Number of reads in raw_data/SRR292770_S1_L001_R1_001.fastq: 5,102,041
Number of reads in raw_data/SRR292770_S1_L001_R2_001.fastq: 5,102,041
Number of reads in raw_data/SRR292862_S2_L001_R1_001.fastq: 5,102,041
Number of reads in raw_data/SRR292862_S2_L001_R2_001.fastq: 5,102,041
Number of reads in total: 31,406,856


In [12]:
! mkdir fastqc_reports

In [19]:
! fastqc -q -t 6 -o fastqc_reports/ raw_data/*

## K-mer profile and genome size estimation

In [11]:
! jellyfish count --help

Usage: jellyfish count [options] file:path+

Count k-mers in fasta or fastq files

Options (default value in (), *required):
 -m, --mer-len=uint32                    *Length of mer
 -s, --size=uint64                       *Initial hash size
 -t, --threads=uint32                     Number of threads (1)
     --sam=PATH                           SAM/BAM/CRAM formatted input file
 -F, --Files=uint32                       Number files open simultaneously (1)
 -g, --generator=path                     File of commands generating fast[aq]
 -G, --Generators=uint32                  Number of generators run simultaneously (1)
 -S, --shell=string                       Shell used to run generator commands ($SHELL or /bin/sh)
 -o, --output=string                      Output file (mer_counts.jf)
 -c, --counter-len=Length in bits         Length bits of counting field (7)
     --out-counter-len=Length in bytes    Length in bytes of counter field in output (4)
 -C, --canonical                         

In [12]:
! mkdir jellyfish_analysis

In [7]:
! jellyfish count -C -t 8 -s 20000000 -m 31 -o jellyfish_analysis/kmer_counts_31.jf raw_data/SRR292678sub_S1_L001_R1_001.fastq